In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("IPL-Batch-Analytics") \
    .getOrCreate()


In [8]:
matches = spark.read.option("header", True).csv("matches.csv")
deliveries = spark.read.option("header", True).csv("deliveries.csv")

print("Matches:", matches.count())
print("Deliveries:", deliveries.count())


Matches: 1095
Deliveries: 260920


In [9]:
#Step 1: Inspect schema (VERY IMPORTANT)
matches.printSchema()
deliveries.printSchema()


root
 |-- id: string (nullable = true)
 |-- season: string (nullable = true)
 |-- city: string (nullable = true)
 |-- date: string (nullable = true)
 |-- match_type: string (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- toss_winner: string (nullable = true)
 |-- toss_decision: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- result: string (nullable = true)
 |-- result_margin: string (nullable = true)
 |-- target_runs: string (nullable = true)
 |-- target_overs: string (nullable = true)
 |-- super_over: string (nullable = true)
 |-- method: string (nullable = true)
 |-- umpire1: string (nullable = true)
 |-- umpire2: string (nullable = true)

root
 |-- match_id: string (nullable = true)
 |-- inning: string (nullable = true)
 |-- batting_team: string (nullable = true)
 |-- bowling_team: string (nullable = true)
 |-- over: string (nullab

In [10]:
matches.show(5)
deliveries.show(5)


+------+-------+----------+----------+----------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+-----------+------------+----------+------+---------+--------------+
|    id| season|      city|      date|match_type|player_of_match|               venue|               team1|               team2|         toss_winner|toss_decision|              winner| result|result_margin|target_runs|target_overs|super_over|method|  umpire1|       umpire2|
+------+-------+----------+----------+----------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+-------------+-----------+------------+----------+------+---------+--------------+
|335982|2007/08| Bangalore|2008-04-18|    League|    BB McCullum|M Chinnaswamy Sta...|Royal Challengers...|Kolkata Knight Ri...|Royal Challengers...|        field|Kolkata Knig

In [11]:
#Import required functions
from pyspark.sql.functions import col


In [13]:
#Cast important columns in deliveries
deliveries_clean = deliveries \
    .withColumn("match_id", col("match_id").cast("int")) \
    .withColumn("over", col("over").cast("int")) \
    .withColumn("ball", col("ball").cast("int")) \
    .withColumn("total_runs", col("total_runs").cast("int")) \
    .withColumn("is_wicket", col("is_wicket").cast("int"))


In [14]:
deliveries_clean.printSchema()


root
 |-- match_id: integer (nullable = true)
 |-- inning: string (nullable = true)
 |-- batting_team: string (nullable = true)
 |-- bowling_team: string (nullable = true)
 |-- over: integer (nullable = true)
 |-- ball: integer (nullable = true)
 |-- batter: string (nullable = true)
 |-- bowler: string (nullable = true)
 |-- non_striker: string (nullable = true)
 |-- batsman_runs: string (nullable = true)
 |-- extra_runs: string (nullable = true)
 |-- total_runs: integer (nullable = true)
 |-- extras_type: string (nullable = true)
 |-- is_wicket: integer (nullable = true)
 |-- player_dismissed: string (nullable = true)
 |-- dismissal_kind: string (nullable = true)
 |-- fielder: string (nullable = true)



In [15]:
#Cast important columns in matches
matches_clean = matches \
    .withColumn("id", col("id").cast("int")) \
    .withColumn("season", col("season").cast("int"))



In [16]:
# just for checking if aggregate functions are working or not
#Total runs scored per match
from pyspark.sql.functions import sum

runs_per_match = deliveries_clean.groupBy("match_id") \
    .agg(sum("total_runs").alias("total_runs"))

runs_per_match.show(5)


+--------+----------+
|match_id|total_runs|
+--------+----------+
|  501247|       253|
|  501269|       295|
|  829705|       338|
|  980905|       323|
| 1254081|       343|
+--------+----------+
only showing top 5 rows



In [17]:
matches_clean.printSchema()


root
 |-- id: integer (nullable = true)
 |-- season: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- date: string (nullable = true)
 |-- match_type: string (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- toss_winner: string (nullable = true)
 |-- toss_decision: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- result: string (nullable = true)
 |-- result_margin: string (nullable = true)
 |-- target_runs: string (nullable = true)
 |-- target_overs: string (nullable = true)
 |-- super_over: string (nullable = true)
 |-- method: string (nullable = true)
 |-- umpire1: string (nullable = true)
 |-- umpire2: string (nullable = true)



In [18]:
from pyspark.sql.functions import year, to_date
matches_clean = matches_clean.withColumn(
    "year",
    year(to_date(col("date"), "yyyy-MM-dd"))
)


In [19]:
matches_clean.select("id", "date", "year").show(5)


+------+----------+----+
|    id|      date|year|
+------+----------+----+
|335982|2008-04-18|2008|
|335983|2008-04-19|2008|
|335984|2008-04-19|2008|
|335985|2008-04-20|2008|
|335986|2008-04-20|2008|
+------+----------+----+
only showing top 5 rows



In [20]:
#STEP 1: Make the result MEANINGFUL (JOIN WITH MATCHES)
#STEP 1.1: Join runs with match details
runs_with_match = runs_per_match.join(
    matches_clean,
    runs_per_match.match_id == matches_clean.id,
    "inner"
)


In [21]:
#STEP 1.2: Select only useful columns
runs_with_match_selected = runs_with_match.select(
    runs_per_match.match_id,
    matches_clean.year,
    matches_clean.team1,
    matches_clean.team2,
    matches_clean.venue,
    runs_per_match.total_runs
)

In [22]:
runs_per_match.show(5)

+--------+----------+
|match_id|total_runs|
+--------+----------+
|  501247|       253|
|  501269|       295|
|  829705|       338|
|  980905|       323|
| 1254081|       343|
+--------+----------+
only showing top 5 rows



In [24]:
#Write match-wise runs to HDFS
runs_per_match.write \
    .mode("overwrite") \
    .parquet("hdfs:///ipl/curated/match_wise_runs")


In [25]:
#STEP 1.3: Order results (deterministic output)
runs_with_match_selected.orderBy("match_id").show(5)



+--------+----+--------------------+--------------------+--------------------+----------+
|match_id|year|               team1|               team2|               venue|total_runs|
+--------+----+--------------------+--------------------+--------------------+----------+
|  335982|2008|Royal Challengers...|Kolkata Knight Ri...|M Chinnaswamy Sta...|       304|
|  335983|2008|     Kings XI Punjab| Chennai Super Kings|Punjab Cricket As...|       447|
|  335984|2008|    Delhi Daredevils|    Rajasthan Royals|    Feroz Shah Kotla|       261|
|  335985|2008|      Mumbai Indians|Royal Challengers...|    Wankhede Stadium|       331|
|  335986|2008|Kolkata Knight Ri...|     Deccan Chargers|        Eden Gardens|       222|
+--------+----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [26]:
#Write detailed match-wise runs to HDFS
runs_with_match_selected.write \
    .mode("overwrite") \
    .parquet("hdfs:///ipl/curated/match_wise_runs_detailed")


In [27]:
#STEP 2: SEASON-WISE (YEAR-WISE) BATCH ANALYTICS
#STEP 2.1: Season-wise total runs
from pyspark.sql.functions import sum

season_wise_runs = runs_with_match_selected.groupBy("year") \
    .agg(sum("total_runs").alias("season_total_runs")) \
    .orderBy("year")

season_wise_runs.show()


+----+-----------------+
|year|season_total_runs|
+----+-----------------+
|2008|            17937|
|2009|            16353|
|2010|            18883|
|2011|            21154|
|2012|            22453|
|2013|            22602|
|2014|            18931|
|2015|            18353|
|2016|            18862|
|2017|            18786|
|2018|            19901|
|2019|            19434|
|2020|            19416|
|2021|            18637|
|2022|            24395|
|2023|            25688|
|2024|            25971|
+----+-----------------+



In [28]:
#STEP 2.2: Write this to HDFS (CURATED LAYER)
season_wise_runs.write \
    .mode("overwrite") \
    .parquet("hdfs:///ipl/curated/season_wise_runs")


In [29]:
#STEP 3: TEAM-WISE BATCH ANALYTICS
#STEP 3.1: Team-wise total runs (COMPUTE)
from pyspark.sql.functions import sum, col

team_wise_runs = deliveries_clean.groupBy("batting_team") \
    .agg(sum("total_runs").alias("team_total_runs")) \
    .orderBy(col("team_total_runs").desc())

team_wise_runs.show(20)


+--------------------+---------------+
|        batting_team|team_total_runs|
+--------------------+---------------+
|      Mumbai Indians|          42176|
|Kolkata Knight Ri...|          39331|
| Chennai Super Kings|          38629|
|Royal Challengers...|          37692|
|    Rajasthan Royals|          34747|
|     Kings XI Punjab|          30064|
| Sunrisers Hyderabad|          29071|
|    Delhi Daredevils|          24296|
|      Delhi Capitals|          14900|
|     Deccan Chargers|          11463|
|        Punjab Kings|           9536|
|      Gujarat Titans|           7757|
|Lucknow Super Giants|           7510|
|       Pune Warriors|           6358|
|       Gujarat Lions|           4862|
|Royal Challengers...|           2930|
|Rising Pune Super...|           2470|
|Rising Pune Super...|           2063|
|Kochi Tuskers Kerala|           1901|
+--------------------+---------------+



In [30]:
#STEP 3.3: TEAM NAME STANDARDIZATION (DATA CLEANING)
from pyspark.sql.functions import when, col

def standardize_team(col_name):
    return (
        when(col(col_name).isin("Royal Challengers Bangalore", "Royal Challengers Bengaluru"),
             "Royal Challengers Bangalore")
        .when(col(col_name).isin("Rising Pune Supergiant", "Rising Pune Supergiants"),
              "Rising Pune Supergiant")
        .when(col(col_name).isin("Delhi Daredevils", "Delhi Capitals"),
              "Delhi Capitals")
        .when(col(col_name).isin("Kings XI Punjab", "Punjab Kings"),
              "Punjab Kings")
        .otherwise(col(col_name))
    )


In [31]:
deliveries_standardized = deliveries_clean \
    .withColumn("batting_team_std", standardize_team("batting_team")) \
    .withColumn("bowling_team_std", standardize_team("bowling_team"))


In [32]:
#STEP 3.4: TEAM-WISE TOTAL RUNS (CLEAN ANALYSIS)
team_wise_runs_clean = deliveries_standardized.groupBy("batting_team_std") \
    .agg(sum("total_runs").alias("team_total_runs")) \
    .orderBy(col("team_total_runs").desc())

team_wise_runs_clean.show(20)


+--------------------+---------------+
|    batting_team_std|team_total_runs|
+--------------------+---------------+
|      Mumbai Indians|          42176|
|Royal Challengers...|          40622|
|        Punjab Kings|          39600|
|Kolkata Knight Ri...|          39331|
|      Delhi Capitals|          39196|
| Chennai Super Kings|          38629|
|    Rajasthan Royals|          34747|
| Sunrisers Hyderabad|          29071|
|     Deccan Chargers|          11463|
|      Gujarat Titans|           7757|
|Lucknow Super Giants|           7510|
|       Pune Warriors|           6358|
|       Gujarat Lions|           4862|
|Rising Pune Super...|           4533|
|Kochi Tuskers Kerala|           1901|
+--------------------+---------------+



In [33]:
team_wise_runs_clean.write \
    .mode("overwrite") \
    .parquet("hdfs:///ipl/curated/team_wise_runs")


In [34]:
#STEP 3.5: WRITE CLEAN TEAM-WISE RUNS TO HDFS
team_wise_runs_clean.write \
    .mode("overwrite") \
    .parquet("hdfs:///ipl/curated/team_wise_runs")


In [35]:
#STEP 4: TEAM-WISE WICKETS (BOWLING ANALYTICS)
#STEP 4.2: Compute team-wise total wickets
from pyspark.sql.functions import sum, col

team_wise_wickets = deliveries_standardized.groupBy("bowling_team_std") \
    .agg(sum("is_wicket").alias("total_wickets")) \
    .orderBy(col("total_wickets").desc())

team_wise_wickets.show(20)


+--------------------+-------------+
|    bowling_team_std|total_wickets|
+--------------------+-------------+
|      Mumbai Indians|         1591|
|      Delhi Capitals|         1498|
|Royal Challengers...|         1494|
| Chennai Super Kings|         1481|
|Kolkata Knight Ri...|         1464|
|        Punjab Kings|         1411|
|    Rajasthan Royals|         1277|
| Sunrisers Hyderabad|         1074|
|     Deccan Chargers|          446|
|      Gujarat Titans|          298|
|Lucknow Super Giants|          264|
|       Pune Warriors|          238|
|Rising Pune Super...|          189|
|       Gujarat Lions|          151|
|Kochi Tuskers Kerala|           74|
+--------------------+-------------+



In [36]:
#STEP 4.3: Write team-wise wickets to HDFS
team_wise_wickets.write \
    .mode("overwrite") \
    .parquet("hdfs:///ipl/curated/team_wise_wickets")

    

In [37]:
#STEP 5: PLAYER-WISE ANALYTICS (BATCH)
#STEP 5.1: PLAYER-WISE TOTAL RUNS (BATSMEN)
from pyspark.sql.functions import sum, col

player_wise_runs = deliveries_clean.groupBy("batter") \
    .agg(sum("batsman_runs").alias("total_runs")) \
    .orderBy(col("total_runs").desc())

player_wise_runs.show(20)


+--------------+----------+
|        batter|total_runs|
+--------------+----------+
|       V Kohli|    8014.0|
|      S Dhawan|    6769.0|
|     RG Sharma|    6630.0|
|     DA Warner|    6567.0|
|      SK Raina|    5536.0|
|      MS Dhoni|    5243.0|
|AB de Villiers|    5181.0|
|      CH Gayle|    4997.0|
|    RV Uthappa|    4954.0|
|    KD Karthik|    4843.0|
|      KL Rahul|    4689.0|
|     AM Rahane|    4642.0|
|  F du Plessis|    4571.0|
|     SV Samson|    4419.0|
|     AT Rayudu|    4348.0|
|     G Gambhir|    4217.0|
|     SR Watson|    3880.0|
|     MK Pandey|    3859.0|
|      SA Yadav|    3594.0|
|    JC Buttler|    3583.0|
+--------------+----------+
only showing top 20 rows



In [38]:
#STEP 5.2: PLAYER-WISE MOST SIXES
from pyspark.sql.functions import count

player_wise_sixes = deliveries_clean \
    .filter(col("batsman_runs") == 6) \
    .groupBy("batter") \
    .agg(count("*").alias("total_sixes")) \
    .orderBy(col("total_sixes").desc())

player_wise_sixes.show(20)


+--------------+-----------+
|        batter|total_sixes|
+--------------+-----------+
|      CH Gayle|        359|
|     RG Sharma|        281|
|       V Kohli|        273|
|AB de Villiers|        253|
|      MS Dhoni|        252|
|     DA Warner|        236|
|    KA Pollard|        224|
|    AD Russell|        209|
|     SV Samson|        206|
|      SK Raina|        204|
|     SR Watson|        190|
|      KL Rahul|        187|
|    RV Uthappa|        182|
|     AT Rayudu|        173|
|  F du Plessis|        166|
|    KD Karthik|        161|
|    JC Buttler|        161|
|     YK Pathan|        161|
|    GJ Maxwell|        160|
|       RR Pant|        154|
+--------------+-----------+
only showing top 20 rows



In [39]:
#STEP 5.3: PLAYER-WISE MOST FOURS
player_wise_fours = deliveries_clean \
    .filter(col("batsman_runs") == 4) \
    .groupBy("batter") \
    .agg(count("*").alias("total_fours")) \
    .orderBy(col("total_fours").desc())

player_wise_fours.show(20)


+--------------+-----------+
|        batter|total_fours|
+--------------+-----------+
|      S Dhawan|        768|
|       V Kohli|        708|
|     DA Warner|        663|
|     RG Sharma|        599|
|      SK Raina|        506|
|     G Gambhir|        492|
|    RV Uthappa|        481|
|     AM Rahane|        479|
|    KD Karthik|        466|
|  F du Plessis|        422|
|AB de Villiers|        414|
|      CH Gayle|        408|
|      KL Rahul|        400|
|      SA Yadav|        385|
|     SR Watson|        377|
|      PA Patel|        365|
|      MS Dhoni|        363|
|     AT Rayudu|        359|
|    JC Buttler|        356|
|     SV Samson|        352|
+--------------+-----------+
only showing top 20 rows



In [40]:
#STEP 5.4: PLAYER-WISE MATCHES PLAYED
from pyspark.sql.functions import col, countDistinct

# Batter appearances
batter_df = deliveries_clean \
    .filter(col("batter").isNotNull()) \
    .filter(col("batter") != "NA") \
    .select(col("batter").alias("player"), "match_id")

# Bowler appearances
bowler_df = deliveries_clean \
    .filter(col("bowler").isNotNull()) \
    .filter(col("bowler") != "NA") \
    .select(col("bowler").alias("player"), "match_id")

# Fielder appearances
fielder_df = deliveries_clean \
    .filter(col("fielder").isNotNull()) \
    .filter(col("fielder") != "NA") \
    .select(col("fielder").alias("player"), "match_id")

# Union all roles
all_players_df = batter_df.union(bowler_df).union(fielder_df)

# Count distinct matches
player_matches = all_players_df.groupBy("player") \
    .agg(countDistinct("match_id").alias("matches_played")) \
    .orderBy(col("matches_played").desc())

player_matches.show(20)



+---------------+--------------+
|         player|matches_played|
+---------------+--------------+
|      RG Sharma|           254|
|       MS Dhoni|           250|
|     KD Karthik|           248|
|        V Kohli|           247|
|      RA Jadeja|           237|
|       S Dhawan|           222|
|       R Ashwin|           210|
|       SK Raina|           201|
|     RV Uthappa|           200|
|      AT Rayudu|           193|
|      PP Chawla|           192|
|      DA Warner|           185|
|     KA Pollard|           184|
| AB de Villiers|           179|
|      SP Narine|           176|
|        B Kumar|           176|
|      AM Rahane|           176|
|      SV Samson|           167|
|Harbhajan Singh|           163|
|      YK Pathan|           163|
+---------------+--------------+
only showing top 20 rows



In [41]:
#STEP 5.5: PLAYER-WISE TOTAL WICKETS (BOWLERS)
from pyspark.sql.functions import col, count

player_wise_wickets = deliveries_clean \
    .filter(col("is_wicket") == 1) \
    .filter(col("dismissal_kind") != "run out") \
    .groupBy("bowler") \
    .agg(count("*").alias("total_wickets")) \
    .orderBy(col("total_wickets").desc())

player_wise_wickets.show(20)


+---------------+-------------+
|         bowler|total_wickets|
+---------------+-------------+
|      YS Chahal|          205|
|      PP Chawla|          192|
|       DJ Bravo|          183|
|        B Kumar|          181|
|       R Ashwin|          181|
|      SP Narine|          180|
|       A Mishra|          174|
|     SL Malinga|          170|
|      JJ Bumrah|          168|
|      RA Jadeja|          160|
|Harbhajan Singh|          150|
|    Rashid Khan|          149|
|       UT Yadav|          144|
| Sandeep Sharma|          137|
|       HV Patel|          135|
|      MM Sharma|          132|
| Mohammed Shami|          127|
|       AR Patel|          123|
|       TA Boult|          121|
|       K Rabada|          120|
+---------------+-------------+
only showing top 20 rows



In [42]:
#5.6: PLAYER-WISE MOST CATCHES (FIELDING)
player_wise_catches = deliveries_clean \
    .filter(col("dismissal_kind") == "caught") \
    .filter(col("fielder").isNotNull()) \
    .filter(col("fielder") != "NA") \
    .groupBy("fielder") \
    .agg(count("*").alias("total_catches")) \
    .orderBy(col("total_catches").desc())

player_wise_catches.show(20)


+--------------+-------------+
|       fielder|total_catches|
+--------------+-------------+
|      MS Dhoni|          152|
|    KD Karthik|          145|
|AB de Villiers|          120|
|       V Kohli|          114|
|      SK Raina|          106|
|      S Dhawan|          100|
|     RG Sharma|          100|
|    KA Pollard|           97|
|     RA Jadeja|           97|
|       WP Saha|           93|
|    RV Uthappa|           92|
|     DA Warner|           88|
|     MK Pandey|           85|
|     SV Samson|           82|
|  F du Plessis|           82|
|     DA Miller|           80|
|      KL Rahul|           78|
|       RR Pant|           75|
|     AM Rahane|           72|
|     Q de Kock|           70|
+--------------+-------------+
only showing top 20 rows



In [43]:
#STEP 5.7: PLAYER-WISE MOST RUN-OUT INVOLVEMENTS
player_wise_runouts = deliveries_clean \
    .filter(col("dismissal_kind") == "run out") \
    .filter(col("fielder").isNotNull()) \
    .filter(col("fielder") != "NA") \
    .groupBy("fielder") \
    .agg(count("*").alias("run_outs")) \
    .orderBy(col("run_outs").desc())

player_wise_runouts.show(20)


+--------------+--------+
|       fielder|run_outs|
+--------------+--------+
|      MS Dhoni|      26|
|     RA Jadeja|      19|
|       V Kohli|      17|
|     SV Samson|      17|
|    KD Karthik|      17|
|AB de Villiers|      13|
|      SK Raina|      13|
|     MK Pandey|      13|
|       WP Saha|      12|
|      DJ Bravo|      12|
|     RG Sharma|      11|
|     SPD Smith|      10|
|      PA Patel|      10|
|     YK Pathan|       9|
|      A Mishra|       9|
|    KA Pollard|       9|
|     DA Warner|       9|
|    RV Uthappa|       9|
|  Ishan Kishan|       9|
|     AM Rahane|       9|
+--------------+--------+
only showing top 20 rows



In [45]:
#STEP 5.8:PLAYER-WISE STUMPINGS (WICKET-KEEPERS)
player_wise_stumpings = deliveries_clean \
    .filter(col("dismissal_kind") == "stumped") \
    .filter(col("fielder").isNotNull()) \
    .filter(col("fielder") != "NA") \
    .groupBy("fielder") \
    .agg(count("*").alias("total_stumpings")) \
    .orderBy(col("total_stumpings").desc())

player_wise_stumpings.show(20)


+--------------+---------------+
|       fielder|total_stumpings|
+--------------+---------------+
|      MS Dhoni|             42|
|    KD Karthik|             37|
|    RV Uthappa|             32|
|       WP Saha|             26|
|       RR Pant|             23|
|      PA Patel|             16|
|  AC Gilchrist|             16|
|     Q de Kock|             16|
|     SV Samson|             16|
|       NV Ojha|             10|
| KC Sangakkara|              9|
|AB de Villiers|              8|
|     H Klaasen|              7|
|     KM Jadhav|              7|
|      MS Bisla|              7|
|    SP Goswami|              7|
|      KL Rahul|              7|
|       AP Tare|              6|
|   BB McCullum|              6|
|     DH Yagnik|              5|
+--------------+---------------+
only showing top 20 rows



In [46]:
# ================================
# STEP 5.9: WRITE ALL PLAYER-WISE ANALYTICS TO HDFS
# ================================

player_wise_runs.write \
    .mode("overwrite") \
    .parquet("hdfs:///ipl/curated/player_wise_runs")

player_wise_fours.write \
    .mode("overwrite") \
    .parquet("hdfs:///ipl/curated/player_wise_fours")

player_wise_sixes.write \
    .mode("overwrite") \
    .parquet("hdfs:///ipl/curated/player_wise_sixes")

player_wise_wickets.write \
    .mode("overwrite") \
    .parquet("hdfs:///ipl/curated/player_wise_wickets")

player_wise_catches.write \
    .mode("overwrite") \
    .parquet("hdfs:///ipl/curated/player_wise_catches")

player_wise_runouts.write \
    .mode("overwrite") \
    .parquet("hdfs:///ipl/curated/player_wise_runouts")

player_matches.write \
    .mode("overwrite") \
    .parquet("hdfs:///ipl/curated/player_matches")

player_wise_stumpings.write \
    .mode("overwrite") \
    .parquet("hdfs:///ipl/curated/player_wise_stumpings")
